In [ ]:
%load_ext lab_black

In [4]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 10000

# Generate predictors
age_60_plus = np.random.binomial(1, 0.3, n_samples)
high_cholesterol = np.random.binomial(1, 0.4, n_samples)
diabetes = np.random.binomial(1, 0.25, n_samples)
preventative_services = np.random.binomial(1, 0.5, n_samples)

# Generate hospital_id (from 1 to 10)
hospital_id = np.random.randint(1, 4, n_samples)

# Initialize the outcome array
er_visit = np.zeros(n_samples, dtype=int)

# Simulate the outcome based on correlations
for i in range(n_samples):
    prob = 0.1  # Base probability of ER visit
    if age_60_plus[i] == 1:
        prob += 0.4
    if high_cholesterol[i] == 1:
        prob += 0.4
    if diabetes[i] == 1:
        prob += 0.2
    if preventative_services[i] == 1:
        prob -= 0.4

    er_visit[i] = np.random.binomial(1, min(max(prob, 0), 1))

# Create a DataFrame
data = pd.DataFrame(
    {
        "Hospital ID": hospital_id,
        "Age 60+": age_60_plus,
        "High Cholesterol": high_cholesterol,
        "Diabetes": diabetes,
        "Preventative Services": preventative_services,
        "ER Visit": er_visit,
    }
)

# Display the first few rows of the dataset
data["Hospital ID"].value_counts()

1    3360
2    3343
3    3297
Name: Hospital ID, dtype: int64

In [5]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import shap

# Prepare the data
X = data.drop(["ER Visit", "Hospital ID"], axis=1)
y = data["ER Visit"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train the XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

# Make predictions (opxtional, to evaluate model)
predictions = model.predict(X_test)

In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import shap

# Assuming 'data' is your DataFrame and is already defined
# ... [Your existing code for data preparation and model training] ...

# Create a Tree explainer
explainer = shap.Explainer(
    model, X_train, model_output="probability", feature_perturbation="interventional"
)

# Calculate SHAP values - this might take some time for larger datasets
shap_values = explainer(data.drop(["ER Visit", "Hospital ID"], axis=1))

 98%|===================| 9822/10000 [00:18<00:00]       

In [7]:
# Assuming shap_values and X have the same order
column_mapping = {
    i: name
    for i, name in enumerate(data.drop(["ER Visit", "Hospital ID"], axis=1).columns)
}

# Rename columns in the DataFrame created from SHAP values
data_shap_pd = pd.DataFrame(shap_values.values).rename(columns=column_mapping)

In [8]:
hospital_pd = pd.DataFrame(hospital_id).rename(columns={0: "Hosptial ID"})

In [9]:
data_no_id_outcome = data.drop(columns=["Hospital ID", "ER Visit"])
data_percentile = data_no_id_outcome.rank(pct=True)

Step: Add back hosptial ids and add unique ids

In [10]:
data_percentile_id = pd.concat([hospital_pd, data_percentile], axis=1)
data_shap_id = pd.concat([hospital_pd, data_shap_pd], axis=1)

In [11]:
import pandas as pd


def median_shap_for_high_percentiles(percentile_df, shap_df, id_col):
    # Initialize a dictionary to store the results
    median_shap_values = {id_col: [], "variable": [], "median_shap": []}

    # Iterate over each ID
    for id_value in percentile_df[id_col].unique():
        # Filter DataFrames for the current ID
        percentile_subdf = percentile_df[percentile_df[id_col] == id_value]
        shap_subdf = shap_df[shap_df[id_col] == id_value]

        # Iterate over each column (except the ID column)
        for col in percentile_df.columns:
            if col == id_col:
                continue

            # Calculate the median of the current column in percentile DataFrame
            median_value = percentile_subdf[col].median()

            # Filter rows where the percentile value is above the median
            rows_above_median = percentile_subdf[
                percentile_subdf[col] > median_value
            ].index

            # Calculate the median SHAP value for these rows
            median_shap = shap_subdf.loc[rows_above_median, col].median()

            # Store the results
            median_shap_values[id_col].append(id_value)
            median_shap_values["variable"].append(col)
            median_shap_values["median_shap"].append(median_shap)

    # Convert the dictionary to a DataFrame and return
    return pd.DataFrame(median_shap_values)

In [12]:
data_percentile_id[data_percentile_id["Hosptial ID"] == 2]["Preventative Services"]

5       0.75005
7       0.25005
8       0.75005
9       0.25005
13      0.75005
         ...   
9989    0.75005
9990    0.25005
9991    0.75005
9995    0.75005
9998    0.25005
Name: Preventative Services, Length: 3343, dtype: float64

In [13]:
# Example usage
result_df = (
    median_shap_for_high_percentiles(
        percentile_df=data_percentile_id, shap_df=data_shap_id, id_col="Hosptial ID"
    )
    .fillna(-0.20)
    .round(2)
)

In [14]:


# Function to add noise
def add_noise(series, noise_level=0.05):
    # Generate random noise
    noise = np.random.uniform(-noise_level, noise_level, series.shape)
    return series + noise

# Adding noise to each feature
result_df['Age 60+'] = add_noise(data['Age 60+'])
result_df['High Cholesterol'] = add_noise(result_df['High Cholesterol'])
result_df['Diabetes'] = add_noise(data['Diabetes'])
result_df['Preventative Services'] = add_noise(result_df['Preventative Services'])

# Note: You might want to ensure that the noisy features still make sense
# For example, binary features should still be 0 or 1 after adding noise
# In such cases, you can round the values or clip them to the [0, 1] range

# Display the first few rows of the dataset with noise
print(result_df.head())


KeyError: 'High Cholesterol'